In [1]:
from transformers import CLIPVisionModel, AutoModel
model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
model = AutoModel.from_pretrained('facebook/dinov2-base')

/home/jth264/.conda/envs/benchmark/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
img = torch.rand(1, 3, 224, 224)

In [3]:
output = model(img)

In [4]:
output.last_hidden_state.shape

torch.Size([1, 257, 768])

In [5]:
output.pooler_output.shape

torch.Size([1, 768])

In [6]:
from fish_benchmark.models import MediaClassifier

In [7]:
model = MediaClassifier(5, 'dino', 'linear')

In [8]:
model(img)

tensor([[ 0.8022,  0.3740, -0.7066, -0.1085, -1.5983]],
       grad_fn=<AddmmBackward0>)

In [9]:
from transformers import VideoMAEModel
import torch
video = torch.rand(1, 16, 3, 224, 224)
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")
output = model(video)

In [10]:
type(output)

transformers.modeling_outputs.BaseModelOutput

In [11]:
output.last_hidden_state.shape

torch.Size([1, 1568, 768])

In [12]:
pooling = output.last_hidden_state.mean(dim=1)
pooling.shape

torch.Size([1, 768])

In [26]:
from torch.nn import MultiheadAttention
import torch.nn as nn
def key_proj(x):
    return nn.Linear(x.shape[-1], 64)(x)
def value_proj(x):
    return nn.Linear(x.shape[-1], 128)(x)

token_embeddings = torch.rand(1, 1568, 768)
key = key_proj(token_embeddings)     # shape: [1, 1568, 64]
value = value_proj(token_embeddings) # shape: [1, 1568, 128]
attention = MultiheadAttention(768, 8, kdim=64, vdim = 128, batch_first=True)
query = torch.rand(1, 1, 768)
attn_output, attn_weights = attention(query, key, value)

In [14]:
attn_output.shape

torch.Size([1, 101, 768])

In [15]:
model.config.hidden_size

768

In [16]:
from fish_benchmark.models import AttentionBlock
import torch

video = torch.rand(1, 16, 3, 224, 224)
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")
output = model(video)
output.last_hidden_state.shape

torch.Size([1, 1568, 768])

In [17]:
output.last_hidden_state.shape

torch.Size([1, 1568, 768])

In [20]:
head = AttentionBlock(embed_dim = model.config.hidden_size, output_dim = 101)
head(output.last_hidden_state).shape

torch.Size([1, 101, 1])